In [1]:
import sys
sys.path.insert(0, '../')
from bibliotecas import base_estudantes, job, magalu, alunos, petrobras, sns, np, pd, plt, yf, px, go, sp, wn, quandl, date, db, geobr, gpd, os
wn.filterwarnings('ignore')
import pmdarima

In [2]:
import pandas_datareader.data as web

In [3]:
import plotly.subplots as sub

In [4]:
acoes = ['ARZZ3.SA', 'CEAB3.SA','AMER3.SA', 'MGLU3.SA']
nome = ['Arezzo', 'C&A', 'Americanas', 'MagaLu']

In [6]:
d = {
    'Acao': acoes,
    'Nome': nome
}

df = pd.DataFrame(d)
df

,Acao,Nome
0,ARZZ3.SA,Arezzo
1,CEAB3.SA,C&A
2,AMER3.SA,Americanas
3,MGLU3.SA,MagaLu


In [7]:
dados = pd.DataFrame()



for i,a in enumerate(acoes):
    info =  web.get_data_yahoo(a, start='2022-1-1')
    dados[a] = info.reset_index().Close
    if i ==0:
        dados['Data'] = info.index

dados.set_index('Data', inplace=True)

dados

,ARZZ3.SA,CEAB3.SA,AMER3.SA,MGLU3.SA
Data,,,,
2022-01-03,72.230003,5.91,31.020000,6.72
2022-01-04,70.099998,5.52,29.959999,6.61
2022-01-05,66.730003,5.33,28.620001,6.42
2022-01-06,67.949997,5.17,29.260000,6.25
2022-01-07,67.410004,5.17,27.700001,6.22
...,...,...,...,...
2022-07-21,74.559998,2.60,16.930000,3.01
2022-07-22,74.129997,2.50,15.930000,2.86
2022-07-25,76.209999,2.57,15.570000,2.79


In [8]:
taxa = {}

for i in dados.columns:
    ini = dados[i].iloc[0]
    fim = dados[i].iloc[-1]
    retorno = ((fim-ini)/ini)*100
    retorno = round(retorno,2)

    taxa[i] = retorno

# retorno de quem comprou no comeco e vendeu agr

taxa


{'ARZZ3.SA': 5.9, 'CEAB3.SA': -53.98, 'AMER3.SA': -51.93, 'MGLU3.SA': -61.01}

In [10]:
taxa = {}

# deslocamento diario 

for i in dados.columns:
    desloc = (dados[i]/dados[i].shift(1))-1
    # desloc.fillna(0, inplace=True)
    taxa[i] = desloc.mean()

taxa

{'ARZZ3.SA': 0.0006500490278228442,
 'CEAB3.SA': -0.004656314135153137,
 'AMER3.SA': -0.004305676501716447,
 'MGLU3.SA': -0.005524336422096703}

In [12]:
# calcular quanto lucrei comprando uma quantidade x de acoes

lucro = {}

qnt = 100

for i in dados.columns:
    ini = dados[i].iloc[0]
    fim = dados[i].iloc[-1]

    saldoI = (ini*qnt)
    saldoF = (fim*qnt)

    diff = round((saldoF-saldoI),2)

    lucro[i] = diff

lucro



{'ARZZ3.SA': 426.0,
 'CEAB3.SA': -319.0,
 'AMER3.SA': -1611.0,
 'MGLU3.SA': -410.0}

In [14]:
taxa = []
diaria = []
lucro = []

qnt = 100

for i in dados.columns:
    ini = dados[i].iloc[0]
    fim = dados[i].iloc[-1]
    retorno = ((fim-ini)/ini)*100
    retorno = round(retorno,2)

    taxa.append(retorno)

    desloc = (dados[i]/dados[i].shift(1))-1
    # desloc.fillna(0, inplace=True)
    diaria.append( desloc.mean())

    saldoI = (ini*qnt)
    saldoF = (fim*qnt)

    diff = round((saldoF-saldoI),2)

    lucro.append(diff)

# retorno de quem comprou no comeco e vendeu agr

df['Taxa'] = taxa
df['Diaria'] = diaria
df['Lucro'] = lucro

df

,Acao,Nome,Taxa,Diaria,Lucro
0,ARZZ3.SA,Arezzo,5.90,0.000650,426.0
1,CEAB3.SA,C&A,-53.98,-0.004656,-319.0
2,AMER3.SA,Americanas,-51.93,-0.004306,-1611.0
3,MGLU3.SA,MagaLu,-61.01,-0.005524,-410.0


In [29]:
f = sub.make_subplots(
    rows=3,
    cols=1,
    subplot_titles=['Taxa', 'Diaria', 'Lucro']
)

f.update_layout(
    title_text = 'Relatorio',
    showlegend= False,
    title = {'y':0.95,'x':0.5}
)


f.add_trace(
    go.Scatter(
        x=df['Nome'],
        y=df['Taxa'],
    ),
    row=1,
    col=1
)
 

f.add_trace(
    go.Bar(
        x=df['Nome'],
        y=df['Diaria'],
    ),
    row=2,
    col=1
)


# f.add_trace(
#     go.Line(
#         x=df['Nome'],
#         y=df['Lucro'],
#     ),
#     row=3,
#     col=1
# )
# f.add_trace(
#     go.Line(
#         x=df['Nome'],
#         y=df['Lucro']*df['Taxa'],
#     ),
#     row=3,
#     col=1
# )

f.add_traces(
    [go.Line(
        x=df['Nome'],
        y=df['Lucro']*df['Taxa'],
    ),
    go.Line(
        x=df['Nome'],
        y=df['Lucro'],
    )
    ],
    rows=[3,3],
    cols=[1,1]
)

/home/luisotavio/.local/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


